In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#Crime dataframe
crime_df = pd.read_csv('2020_FBI_Crime_Statistics.csv')
crime_df = crime_df.fillna(0) #fill in Nan with 0's
crime_df['Population'] = crime_df['Population'].astype('int64') #changing to int
crime_df['Arson'] = crime_df['Arson'].astype('int64') #changing to int
crime_df['State'] = crime_df['State'].astype('str') #changing to str
crime_df['City'] = crime_df['City'].astype('str') #changing to str

#Zip Code data frame
zip_df = pd.read_csv('VA_MD_DC_Zips.csv')

#Merging crime and zip DataFrames. Not that not every city in zip has reported crime data to FBI data base
crime_df_merge = pd.merge(zip_df, crime_df, on='City', how = 'left')
crime_df_merge = crime_df_merge.fillna(0) #filling Nan with 0's
crime_df_merge['Population'] = crime_df_merge['Population'].astype('int64')
crime_df_merge['Violent'] = crime_df_merge['Violent'].astype('int64')
crime_df_merge['Murder'] = crime_df_merge['Murder'].astype('int64')
crime_df_merge['Rape'] = crime_df_merge['Rape'].astype('int64')
crime_df_merge['Robbery'] = crime_df_merge['Robbery'].astype('int64')
crime_df_merge['Assault'] = crime_df_merge['Assault'].astype('int64')
crime_df_merge['Property'] = crime_df_merge['Property'].astype('int64')
crime_df_merge['Burglary'] = crime_df_merge['Burglary'].astype('int64')
crime_df_merge['Theft'] = crime_df_merge['Theft'].astype('int64')
crime_df_merge['Vehicle'] = crime_df_merge['Vehicle'].astype('int64')
crime_df_merge['Arson'] = crime_df_merge['Arson'].astype('int64')
crime_df_merge['Total Crimes'] = crime_df_merge['Total Crimes'].astype('int64') 

#setting index to Zip Code 
crime_df_merge.set_index('Zip Code', inplace=True) 

#setting cities without reported crime to the average of the population reported by FBI
crime_df_merge.loc[crime_df_merge['Crime Index']==0, ['Crime Index']] = 4.46 

desired_zip = 20001  #input of zip
result = crime_df_merge.loc[desired_zip, ["Crime Index"]]
result

Crime Index    10.45
Name: 20001, dtype: object